In [31]:
import pandas as pd
import re
import pickle
import string

In [32]:
Dataset = 'Clotho' # or 'Clotho'

In [33]:
def repair_caption(caption) :
    caption = caption.replace(',', ' , ')
    caption = re.sub(' +', ' ', caption)
    caption = caption.replace(' ,', ',')

    return caption

In [34]:
def get_word_dictionary(caption_list) :
    word_dictionary = {}

    for caption in caption_list :

        caption = caption.lower()
        
        # 쉼표를 '안녕,하세요'와 같이 쓰는 사례가 많더라
        # 얘들을 다 교정해준다 
        # 교정========================================
        caption = caption.replace(',', ' , ') 
        # 공백 줄이기
        caption = re.sub(' +', ' ', caption)
        caption = caption.replace(' ,', ',')
        # 교정========================================

        # For ACT Tokenizer
        # caption = re.sub(r'\s([,.!?;:"](?:\s|$))', r'\1', caption).replace('  ', ' ')
        # caption = re.sub('[,.!?;:\"]', ' ', caption).replace('  ', ' ')

        # For Clotho tokenizer
        # caption = caption.translate(str.maketrans('', '', string.punctuation))

        # For Custom Tokenizer1
        # caption = re.sub(r'[.]', '', caption)
        # caption += '.'

        # For Custom Tokenizer2
        caption = re.sub(r'[.]', '', caption)

        word_list = caption.strip().split(' ')

        for word in word_list : 
            
            # Custom Tokenizer2의 경우 사용================================
            if word in word_dictionary: 
                if word[-1] == ',' :
                    temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
                    word_dictionary[temp_word_wo_rest] += 1
                    word_dictionary[','] += 1 # 쉼표를 따로 추가
                else :
                    word_dictionary[word] += 1
            else : 
                if word[-1] == ',' :
                    temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
                    word_dictionary[temp_word_wo_rest] = 1
                    word_dictionary[','] = 1
                else :
                    word_dictionary[word] = 1
            # Custom Tokenizer2의 경우 사용================================

            # if word in word_dictionary: 
            #     word_dictionary[word] += 1
            # else : 
            #     word_dictionary[word] = 1
                
    return sorted(word_dictionary.items(), key = lambda item: item[1], reverse = True)

In [35]:
caption_list_audiocaps = []

train_csv = pd.read_csv('train.csv')

captions = train_csv['caption'].to_list()
for caption in captions:

    edited_caption = repair_caption(caption)

    # if edited_caption != caption :
    #     print("before repair :", caption)
    #     print("after repair :", edited_caption)
    #     print()

    caption_list_audiocaps.append(caption)


In [36]:
word_dictionary_sorted = get_word_dictionary(caption_list_audiocaps) 

word_Dict = []

# 단어만 담기
# 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
for key, value in word_dictionary_sorted :
    word_Dict.append(key)

In [37]:
# GPT-2는 vocab의 맨 첫 idx의 값을 "!"로 했다. 나도 그렇게 해야겠다.
# 그리고 GPT-2는 문장의 마무리라 할 수 있는 마침표('.')의 idx를 13으로 했다. 나는 문장의 마무리를 <eos>로 해야겠다. 
word_Dict.insert(0, '!')
word_Dict.insert(13, '<eos>')
word_Dict.append('<unk>')

In [38]:
# Clotho Dataset에만 있는 word들을 추가해줌
caption_list_clotho = []
if Dataset == 'Clotho' :
    train_csv = pd.read_csv('clotho_captions_development.csv')

    for i in range(5) :
        col_str = 'caption_' + str(i + 1)
    
        captions = train_csv[col_str].to_list()
        for caption in captions:

            edited_caption = repair_caption(caption)

            # if edited_caption != caption :
            #     print("before repair :", caption)
            #     print("after repair :", edited_caption)
            #     print()

            caption_list_clotho.append(caption)

    word_dictionary_sorted = get_word_dictionary(caption_list_clotho)
    # 단어만 담기
    # 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
    not_in_count = 0
    for key, value in word_dictionary_sorted :

        if key not in word_Dict :
            word_Dict.append(key)
            not_in_count +=1
    
    print(str(not_in_count), "words not in audiocaps vocab")

1440 words not in audiocaps vocab


In [39]:
file_name = None
if Dataset == 'AudioCaps' :
    file_name = 'AudioCaps_vocabulary_'
elif Dataset == 'Clotho' :
    file_name = 'Clotho_vocabulary_'

file_name += (str(len(word_Dict)) + '.pickle')

with open(file_name, 'wb') as f:
    pickle.dump(word_Dict, f, pickle.HIGHEST_PROTOCOL)

In [40]:
len(word_Dict)

6524